# Training the model to run on Lichess data
## Some pre-requisites if running on Google Collab
If not running on Google collab do not run these next two cells!

In [ ]:
# Install the only dependency not available from collab directly
!pip install chess

# Get imported files from repo
!git clone -b rl-setup https://github.com/owenjaques/chessbot.git
!mv chessbot chessbot-repo
!mv chessbot-repo/src/chessbot .
!rm chessbot-repo -r

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
weights_directory = '/content/gdrive/MyDrive/chessbot_weights/'
print(f'Saving weights to {weights_directory}')

## Get the data
This compression format is really nice, so you can cancel this cell whenever you want and all the games that were downloaded will be maintained.

In [ ]:
!wget https://database.lichess.org/standard/lichess_db_standard_rated_2023-02.pgn.zst

## Decompress the Data

In [ ]:
!apt install zstd
!pzstd -d lichess_db_standard_rated_2023-02.pgn.zst

## Play the games from the data
For this section we want to play the games so we can translate them into model inputs which we can train on.

In [ ]:
import chess.pgn
import numpy as np
from collections import deque
from chessbot.model_input import ModelInput

X_all = deque()
y_all = deque()

MAX_GAMES = 100000

with open('lichess_db_standard_rated_2023-02.pgn') as pgn:
    game_count = 0
    game = chess.pgn.read_game(pgn)

    while game is not None and game_count < MAX_GAMES:
        result = game.headers['Result']
        
        # Only train on game played to completion that were not draws
        if game.headers['Termination'] == 'Normal' and result in ['1-0', '0-1']:
            print(f'\rProcessing game {game_count}/{MAX_GAMES}', end='')
            X = []
            
            board = game.board()
            for move in game.mainline_moves():
                board.push(move)
                X.append(ModelInput(board).get_input())

            X = np.array(X)
            y = np.empty_like(X)
            
            discount_factor = 0.95
            y_reversed_indices = np.linspace(len(y) - 1, 0, num=len(y))
            y = 1 * discount_factor**y_reversed_indices

            if result == '0-1':
                y = -y

            # Scale the labels to be between 0 and 1 instead of -1 and 1
            y = (y + 1) / 2

            # Save the data
            if X is not None and y is not None:
                try:
                    X_all.extend(X)
                    y_all.extend(y)
                except:
                    pass

        # Get the next game
        game_count += 1
        game = chess.pgn.read_game(pgn)

        # Save current game data every 10000 games
        if game_count % 10000 == 0:
            X = np.array(X_all)
            y = np.array(y_all)
            np.savez_compressed(f'{weights_directory}games_data.npz', X=X, y=y)

In [ ]:
import numpy as np
data = np.load(f'{weights_directory}games_data.npz')
X = data['X']
y = data['y']

## Our model
We will initially be using a MLP Regression model set up with the default parameters from scikit-learn's MLP Regression model since it seems like a solid place to start. After some trial and error, a second Dense layer for the model was added to hopefully capture a bit more complexity.

In [ ]:
from tensorflow import keras

model = keras.Sequential([
	keras.layers.Dense(1024, activation='sigmoid'),
	keras.layers.Dense(1024, activation='sigmoid'),
	keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='mse'
)

## Training the model

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='loss', restore_best_weights=True)
model.fit(X, y, epochs=10, batch_size=128, callbacks=[callback])
model.save(f'{weights_directory}lichess_trained_model')

## Why not play a game after all that training?

In [ ]:
import chess
import numpy as np
import time
from IPython.display import clear_output
from chessbot.chessbot import ChessBot

def play_game(model, exploration_rate=0.0, should_visualise=False):
	white = ChessBot(model, chess.WHITE, exploration_rate)
	black = ChessBot(model, chess.BLACK, exploration_rate)

	board = chess.Board()

	if should_visualise:
		display(board)

	while not board.is_game_over(claim_draw=True):
		board.push(black.move(board) if board.turn == chess.BLACK else white.move(board))

		if should_visualise:
			clear_output(wait=True)
			display(board)
			time.sleep(0.5)

	return board.outcome(claim_draw=True).result()
 
play_game(model, should_visualise=True)